In [1]:
from modules.read import Read
import pandas as pd

In [2]:
extract = Read()

## Lectura S3: Capa Landing

In [4]:
categories_df = extract.s3_read('retail-multisource-pipeline', 'landing/categories.csv')
customers_df = extract.s3_read('retail-multisource-pipeline', 'landing/customers.csv')
departments_df = extract.s3_read('retail-multisource-pipeline', 'landing/departments.csv')
order_items_df = extract.s3_read('retail-multisource-pipeline', 'landing/order_items.csv')
orders_df = extract.s3_read('retail-multisource-pipeline', 'landing/orders.csv')
products_df = extract.s3_read('retail-multisource-pipeline', 'landing/products.csv')

# Transformación

## Enriquecimiento 

### Cagetories 

In [4]:
categories_df.head()

,category_id,category_department_id,category_name
0,1,2,Football
1,2,2,Soccer
2,3,2,Baseball & Softball
3,4,2,Basketball
4,5,2,Lacrosse


In [5]:
## Categories + Departments
categories_df["category_name"] = categories_df["category_name"].str.strip().str.lower()
categories_df = categories_df.merge(departments_df, left_on="category_department_id", right_on="department_id", how="left")

In [6]:
categories_df.head()

,category_id,category_department_id,category_name,department_id,department_name
0,1,2,football,2.0,Fitness
1,2,2,soccer,2.0,Fitness
2,3,2,baseball & softball,2.0,Fitness
3,4,2,basketball,2.0,Fitness
4,5,2,lacrosse,2.0,Fitness


### customers

In [7]:
customers_df.head()

,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
0,1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
1,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725
3,4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
4,5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,00725


In [8]:
## Customers
customers_df["customer_fullname"] = customers_df["customer_fname"].str.strip().str.title() + " " + customers_df["customer_lname"].str.strip().str.title()

In [9]:
customers_df.head()

,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode,customer_fullname
0,1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521,Richard Hernandez
1,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,Mary Barrett
2,3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725,Ann Smith
3,4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069,Mary Jones
4,5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,00725,Robert Hudson


### Products

In [10]:
products_df.head()

,product_id,product_category_id,product_name,product_description,product_price,product_image
0,1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+F...
1,2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+M...
2,3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+M...
3,4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+M...
4,5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+...


In [11]:
## Products + Categories
products_df = products_df.merge(categories_df, left_on="product_category_id", right_on="category_id", how="left")
products_df["product_price"] = products_df["product_price"].astype(float)
products_df["product_image_valid"] = products_df["product_image"].apply(lambda x: str(x).startswith("http"))

In [12]:
products_df.head()

,product_id,product_category_id,product_name,product_description,product_price,product_image,category_id,category_department_id,category_name,department_id,department_name,product_image_valid
0,1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+F...,2.0,2.0,soccer,2.0,Fitness,True
1,2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+M...,2.0,2.0,soccer,2.0,Fitness,True
2,3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+M...,2.0,2.0,soccer,2.0,Fitness,True
3,4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+M...,2.0,2.0,soccer,2.0,Fitness,True
4,5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+...,2.0,2.0,soccer,2.0,Fitness,True


### order_items

In [13]:
order_items_df.head()

,order_item_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price
0,1,1,957,1,299.98,299.98
1,2,2,1073,1,199.99,199.99
2,3,2,502,5,250.00,50.00
3,4,2,403,1,129.99,129.99
4,5,4,897,2,49.98,24.99


In [14]:
## Order Items + Products
order_items_df["order_item_total"] = order_items_df["order_item_quantity"] * order_items_df["order_item_product_price"]
order_items_df = order_items_df.merge(products_df, left_on="order_item_product_id", right_on="product_id", how="left")


In [15]:
order_items_df.head()

,order_item_id,order_item_order_id,order_item_product_id,order_item_quantity,order_item_subtotal,order_item_product_price,order_item_total,product_id,product_category_id,product_name,product_description,product_price,product_image,category_id,category_department_id,category_name,department_id,department_name,product_image_valid
0,1,1,957,1,299.98,299.98,299.98,957,43,Diamondback Women's Serene Classic Comfort Bi,,299.98,http://images.acmesports.sports/Diamondback+Wo...,43.0,7.0,camping & hiking,7.0,Fan Shop,True
1,2,2,1073,1,199.99,199.99,199.99,1073,48,Pelican Sunstream 100 Kayak,,199.99,http://images.acmesports.sports/Pelican+Sunstr...,48.0,7.0,water sports,7.0,Fan Shop,True
2,3,2,502,5,250.00,50.00,250.00,502,24,Nike Men's Dri-FIT Victory Golf Polo,,50.00,http://images.acmesports.sports/Nike+Men%27s+D...,24.0,5.0,women's apparel,5.0,Golf,True
3,4,2,403,1,129.99,129.99,129.99,403,18,Nike Men's CJ Elite 2 TD Football Cleat,,129.99,http://images.acmesports.sports/Nike+Men%27s+C...,18.0,4.0,men's footwear,4.0,Apparel,True
4,5,4,897,2,49.98,24.99,49.98,897,40,Team Golf New England Patriots Putter Grip,,24.99,http://images.acmesports.sports/Team+Golf+New+...,40.0,6.0,accessories,6.0,Outdoors,True


---

### orders

In [16]:
orders_df.head()

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25,11599,CLOSED
1,2,2013-07-25,256,PENDING_PAYMENT
2,3,2013-07-25,12111,COMPLETE
3,4,2013-07-25,8827,CLOSED
4,5,2013-07-25,11318,COMPLETE


In [17]:
## Orders + Customers + Order Items
orders_df["order_date"] = pd.to_datetime(orders_df["order_date"], errors="coerce")
orders_df["order_month"] = orders_df["order_date"].dt.month
orders_df["order_year"] = orders_df["order_date"].dt.year

In [18]:
orders_df.head()

,order_id,order_date,order_customer_id,order_status,order_month,order_year
0,1,2013-07-25,11599,CLOSED,7,2013
1,2,2013-07-25,256,PENDING_PAYMENT,7,2013
2,3,2013-07-25,12111,COMPLETE,7,2013
3,4,2013-07-25,8827,CLOSED,7,2013
4,5,2013-07-25,11318,COMPLETE,7,2013


---

In [19]:
# Total de la orden
order_totals = order_items_df.groupby("order_item_order_id")["order_item_total"].sum().reset_index()
order_totals.rename(columns={"order_item_order_id": "order_id", "order_item_total": "order_total"}, inplace=True)

orders_df = orders_df.merge(order_totals, on="order_id", how="left")
orders_df = orders_df.merge(customers_df, left_on="order_customer_id", right_on="customer_id", how="left")


In [20]:
orders_df.head()

,order_id,order_date,order_customer_id,order_status,order_month,order_year,order_total,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode,customer_fullname
0,1,2013-07-25,11599,CLOSED,7,2013,299.98,11599,Mary,Malone,XXXXXXXXX,XXXXXXXXX,8708 Indian Horse Highway,Hickory,NC,28601,Mary Malone
1,2,2013-07-25,256,PENDING_PAYMENT,7,2013,579.98,256,David,Rodriguez,XXXXXXXXX,XXXXXXXXX,7605 Tawny Horse Falls,Chicago,IL,60625,David Rodriguez
2,3,2013-07-25,12111,COMPLETE,7,2013,NaN,12111,Amber,Franco,XXXXXXXXX,XXXXXXXXX,8766 Clear Prairie Line,Santa Cruz,CA,95060,Amber Franco
3,4,2013-07-25,8827,CLOSED,7,2013,699.85,8827,Brian,Wilson,XXXXXXXXX,XXXXXXXXX,8396 High Corners,San Antonio,TX,78240,Brian Wilson
4,5,2013-07-25,11318,COMPLETE,7,2013,1129.86,11318,Mary,Henry,XXXXXXXXX,XXXXXXXXX,3047 Silent Embers Maze,Caguas,PR,00725,Mary Henry


## KPIS

In [21]:
# Ventas totales por mes
ventas_mes_df = orders_df.groupby(["order_year", "order_month"])["order_total"].sum().reset_index()
ventas_mes_df.head()

,order_year,order_month,order_total
0,2013,7,764782.19
1,2013,8,2828658.70
2,2013,9,2934527.27
3,2013,10,2624600.61
4,2013,11,3168656.03


In [22]:
# Top 5 productos más vendidos
top_productos_df = order_items_df.groupby("product_name")["order_item_quantity"].sum().reset_index().sort_values(by="order_item_quantity", ascending=False).head(5)
top_productos_df.head()

,product_name,order_item_quantity
60,Perfect Fitness Perfect Rip Deck,73698
48,Nike Men's Dri-FIT Victory Golf Polo,62956
56,O'Brien Men's Neoprene Life Vest,57803
50,Nike Men's Free 5.0+ Running Shoe,36680
85,Under Armour Girls' Toddler Spine Surge Runni,31735


In [23]:
# Ticket promedio
ticket_promedio_df = orders_df["order_total"].mean()
ticket_promedio_df = pd.DataFrame({"ticket_promedio": [ticket_promedio_df]})
ticket_promedio_df.head()

,ticket_promedio
0,597.632288


In [24]:
# Clientes recurrentes
clientes_recurrentes_df = orders_df.groupby("customer_id")["order_id"].nunique().reset_index()
clientes_recurrentes_df = clientes_recurrentes_df[clientes_recurrentes_df["order_id"] > 1]
clientes_recurrentes_df.head()

,customer_id,order_id
1,2,4
2,3,7
3,4,6
4,5,4
5,6,4


In [25]:
# Ventas por estado
ventas_estado_df = orders_df.groupby("customer_state")["order_total"].sum().reset_index()
ventas_estado_df.head()

,customer_state,order_total
0,AL,6401.40
1,AR,29241.61
2,AZ,566459.29
3,CA,5542723.00
4,CO,358310.60
